In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometriosis"
cohort = "GSE75427"

# Input paths
in_trait_dir = "../../input/GEO/Endometriosis"
in_cohort_dir = "../../input/GEO/Endometriosis/GSE75427"

# Output paths
out_data_file = "../../output/preprocess/Endometriosis/GSE75427.csv"
out_gene_data_file = "../../output/preprocess/Endometriosis/gene_data/GSE75427.csv"
out_clinical_data_file = "../../output/preprocess/Endometriosis/clinical_data/GSE75427.csv"
json_path = "../../output/preprocess/Endometriosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression profiles in decidualized and non-decidualized endometriotic cyst stromal cells (ECSCs) and normal endometrial stromal cells (NESCs)"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: endometriotic cyst stromal cells'], 1: ['gender: Female'], 2: ['age: 34y', 'age: 42y', 'age: 30y', 'age: 28y'], 3: ['treatment: 12d 10% charcoal-stripped heat-inactivated FBS', 'treatment: 12d dibutyryl-cAMP and dienogest']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the title mentioning "Expression profiles", this dataset likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Endometriosis), we can see "cell type: proliferative phase normal endometrium" in row 0
# Row 0 likely distinguishes between normal and endometriotic cells
trait_row = 0

# For gender, we see "gender: Female" in row 1, but it appears to be constant (only Female)
# Since there's only one unique value, we consider it not available
gender_row = None

# For age, we see "age: 37y", "age: 47y", etc. in row 2
age_row = 2

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert cell type to binary where 1 indicates endometriotic cells and 0 indicates normal cells."""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Based on the title, we're comparing normal endometrial stromal cells (NESCs)
    # vs endometriotic cyst stromal cells (ECSCs)
    if 'normal' in value.lower():
        return 0  # Normal cells
    elif 'endometrio' in value.lower():
        return 1  # Endometriotic cells
    else:
        return None  # Unknown

def convert_age(value):
    """Convert age to continuous numeric value."""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract numeric age, typically formatted as "XXy" (XX years)
    if 'y' in value:
        try:
            age = int(value.replace('y', '').strip())
            return age
        except ValueError:
            pass
    
    return None  # If conversion fails

def convert_gender(value):
    """
    Convert gender to binary (0 for female, 1 for male).
    Not used in this dataset as gender is constant.
    """
    # This function is included for completeness but not used since gender_row = None
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM1954898': [1.0, 34.0], 'GSM1954899': [1.0, 42.0], 'GSM1954900': [1.0, 30.0], 'GSM1954901': [1.0, 28.0], 'GSM1954902': [1.0, 34.0], 'GSM1954903': [1.0, 42.0], 'GSM1954904': [1.0, 30.0], 'GSM1954905': [1.0, 28.0]}
Clinical data saved to ../../output/preprocess/Endometriosis/clinical_data/GSE75427.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 63
Header line: "ID_REF"	"GSM1954898"	"GSM1954899"	"GSM1954900"	"GSM1954901"	"GSM1954902"	"GSM1954903"	"GSM1954904"	"GSM1954905"
First data line: "A_23_P100001"	354.3793375	172.500875	58.17458	89.16528875	1994.738375	146.5653413	39.38974125	28.5603025
Index(['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074',
       'A_23_P100127', 'A_23_P100141', 'A_23_P100189', 'A_23_P100196',
       'A_23_P100203', 'A_23_P100220', 'A_23_P100240', 'A_23_P10025',
       'A_23_P100292', 'A_23_P100315', 'A_23_P100326', 'A_23_P100344',
       'A_23_P100355', 'A_23_P100386', 'A_23_P100392', 'A_23_P100420'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers don't appear to be standard human gene symbols
# They have a format like "A_19_P00315452" which looks like probe IDs from a microarray platform
# These will need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'SPOT_ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# Examining the structure of gene IDs in both datasets
print("Gene IDs in gene expression data (first 5):")
print(gene_data.index[:5])

# Let's look at more rows of the gene annotation to find the matching columns
print("\nFurther examination of gene annotation (10 more rows):")
print(gene_annotation.iloc[10:20][['ID', 'GENE_SYMBOL']].head(10))

# Try to find any rows with non-null gene symbols
print("\nSample rows with gene symbols (if any):")
symbol_sample = gene_annotation[gene_annotation['GENE_SYMBOL'].notna()].head(5)
print(symbol_sample[['ID', 'GENE_SYMBOL']])

# Check which ID format in the annotation matches our expression data
# Since the standard gene_data IDs look like A_19_P00315452, we need to find the matching pattern
import re

# Find the first few rows where ID matches our expression data pattern
pattern = r'A_\d+_P\d+'
matching_rows = gene_annotation[gene_annotation['ID'].str.contains(pattern, na=False)].head(5)
print("\nRows with matching ID pattern:")
print(matching_rows[['ID', 'GENE_SYMBOL']])

# For probe-gene mapping, we need to determine which columns to use
# Based on the column names, 'ID' should contain probe IDs and 'GENE_SYMBOL' should contain gene symbols
# Let's confirm if there are any rows with both values
valid_mapping_rows = gene_annotation[(gene_annotation['ID'].notna()) & 
                                    (gene_annotation['GENE_SYMBOL'].notna())].head(5)
print("\nRows with both ID and GENE_SYMBOL:")
print(valid_mapping_rows[['ID', 'GENE_SYMBOL']])

# If GENE_SYMBOL is mostly empty, check other potential gene identifier columns
potential_gene_cols = ['GENE', 'GENE_NAME', 'REFSEQ', 'GB_ACC']
for col in potential_gene_cols:
    valid_rows = gene_annotation[(gene_annotation['ID'].notna()) & 
                                (gene_annotation[col].notna())].head(5)
    if not valid_rows.empty:
        print(f"\nRows with both ID and {col}:")
        print(valid_rows[['ID', col]])

# Based on the above analysis, create the mapping dataframe
# Assuming we've identified the correct columns
prob_col = 'ID'  # Column with probe IDs
gene_col = 'GENE_SYMBOL'  # Column with gene symbols (adjust if needed based on results)

# Get the mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print("\nGene mapping dataframe (first 5 rows):")
print(mapping_df.head())

# Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print("\nGene expression data after mapping (first 5 genes):")
print(gene_data.head())

# If the mapping has very few valid entries (or none), we might need to try an alternative approach
# Check the mapping statistics
mapped_count = len(gene_data)
print(f"\nNumber of genes after mapping: {mapped_count}")

# If the mapping resulted in very few genes, try an alternative column
if mapped_count < 100:
    print("Poor mapping results. Trying alternative gene column...")
    # Try using 'GENE' instead of 'GENE_SYMBOL'
    gene_col_alt = 'GENE'
    mapping_df_alt = get_gene_mapping(gene_annotation, prob_col, gene_col_alt)
    gene_data = apply_gene_mapping(gene_data, mapping_df_alt)
    print(f"Number of genes after alternative mapping: {len(gene_data)}")

# Save the gene expression data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene IDs in gene expression data (first 5):
Index(['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074',
       'A_23_P100127'],
      dtype='object', name='ID')

Further examination of gene annotation (10 more rows):
              ID GENE_SYMBOL
10     (-)3xSLv1         NaN
11  A_23_P100001     FAM174B
12  A_23_P100022        SV2B
13  A_23_P100056      RBPMS2
14  A_23_P100074        AVEN
15  A_23_P100127       CASC5
16  A_23_P100141        UNKL
17  A_23_P100189        PRM1
18  A_23_P100196       USP10
19  A_23_P100203       HSBP1

Sample rows with gene symbols (if any):
              ID GENE_SYMBOL
11  A_23_P100001     FAM174B
12  A_23_P100022        SV2B
13  A_23_P100056      RBPMS2
14  A_23_P100074        AVEN
15  A_23_P100127       CASC5



Rows with matching ID pattern:
              ID GENE_SYMBOL
11  A_23_P100001     FAM174B
12  A_23_P100022        SV2B
13  A_23_P100056      RBPMS2
14  A_23_P100074        AVEN
15  A_23_P100127       CASC5

Rows with both ID and GENE_SYMBOL:
              ID GENE_SYMBOL
11  A_23_P100001     FAM174B
12  A_23_P100022        SV2B
13  A_23_P100056      RBPMS2
14  A_23_P100074        AVEN
15  A_23_P100127       CASC5

Rows with both ID and GENE:
              ID    GENE
11  A_23_P100001  400451
12  A_23_P100022    9899
13  A_23_P100056  348093
14  A_23_P100074   57099
15  A_23_P100127   57082

Rows with both ID and GENE_NAME:
              ID                                      GENE_NAME
11  A_23_P100001  family with sequence similarity 174, member B
12  A_23_P100022               synaptic vesicle glycoprotein 2B
13  A_23_P100056   RNA binding protein with multiple splicing 2
14  A_23_P100074        apoptosis, caspase activation inhibitor
15  A_23_P100127              cancer susceptibility


Gene mapping dataframe (first 5 rows):
              ID     Gene
11  A_23_P100001  FAM174B
12  A_23_P100022     SV2B
13  A_23_P100056   RBPMS2
14  A_23_P100074     AVEN
15  A_23_P100127    CASC5

Gene expression data after mapping (first 5 genes):
           GSM1954898   GSM1954899   GSM1954900    GSM1954901   GSM1954902  \
Gene                                                                         
A1BG      3028.378695  2731.904201  3157.886390   3028.031645  3820.889868   
A1BG-AS1   852.177600   601.155425   758.254475   1017.400850   803.633850   
A1CF        13.638512    10.773817    15.022629     10.245584    21.209031   
A2LD1     1528.978615  1301.985750  3653.101250   2065.590500  1868.458601   
A2M       1702.062389  4474.020852  3434.653770  13126.539044   213.549137   

           GSM1954903   GSM1954904   GSM1954905  
Gene                                             
A1BG      3227.197977  3969.369038  5763.236649  
A1BG-AS1   550.132250   894.482050  1676.084700  
A1CF

Gene expression data saved to ../../output/preprocess/Endometriosis/gene_data/GSE75427.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create clinical features directly from clinical_data using the conversion functions defined earlier
clinical_features_df = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from monocytes of rheumatoid arthritis patients, with osteoporosis status included in comorbidity information."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Endometriosis/gene_data/GSE75427.csv
Clinical data saved to ../../output/preprocess/Endometriosis/clinical_data/GSE75427.csv
Linked data shape: (8, 19449)


Quartiles for 'Endometriosis':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Endometriosis' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 29.5
  50% (Median): 32.0
  75%: 36.0
Min: 28.0
Max: 42.0
The distribution of the feature 'Age' in this dataset is fine.

Data was determined to be unusable and was not saved
